In [1]:
import warnings; warnings.filterwarnings("ignore")
from keras.datasets import mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial
import keras.backend as K
import matplotlib.pyplot as plt
import sys
import numpy as np

Using TensorFlow backend.


In [2]:
class RandomWeightedAverage(_Merge):           # inherit a layer (constructor not need args)
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.n_critic = 5                       # recommended in paper
        optimizer = RMSprop(lr=0.00005)         # recommended in paper

        self.generator = self.build_generator() # generator
        self.critic = self.build_critic()       # discriminator 

        # discriminator 
        self.generator.trainable = False # Freeze generator's layers while training critic
        real_img = Input(shape=self.img_shape)
        z_disc = Input(shape=(self.latent_dim,)) # Noise input
        fake_img = self.generator(z_disc)        # generate fake image # tensor(?,28,28,1)
        fake = self.critic(fake_img)             # preidct whether fake # tensor(?,1)
        valid = self.critic(real_img)            # predict whether valid # tensor(?,1)
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
            # input real_img,fake_img into the RWA layer and output interpolated_img
        validity_interpolated = self.critic(interpolated_img) # predict whether valid # tsr(32,1)
        self.critic_model = Model(inputs=[real_img, z_disc], outputs=\
            [valid, fake, validity_interpolated])
        partial_gp_loss = partial(self.gradient_penalty_loss, averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names
        self.critic_model.compile(loss=[self.wasserstein_loss, self.wasserstein_loss, \
            partial_gp_loss], optimizer=optimizer, loss_weights=[1, 1, 10])
        
        # generator
        self.critic.trainable = False
        self.generator.trainable = True
        z_gen = Input(shape=(self.latent_dim,)) # Sampled noise for input to generator
        img = self.generator(z_gen)             # Generate images based of noise
        valid = self.critic(img)                # Discriminator determines validity
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)

    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        # Computes gradient penalty based on prediction and weighted real / fake samples
        gradients = K.gradients(y_pred, averaged_samples)[0]
        gradients_sqr = K.square(gradients) # compute the euclidean norm by squaring
        gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
        gradient_l2_norm = K.sqrt(gradients_sqr_sum) # summing over the rows and sqrt
        gradient_penalty = K.square(1 - gradient_l2_norm)
        return K.mean(gradient_penalty)

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))
        model.summary()
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        return Model(noise, img)

    def build_critic(self):
        model = Sequential()
        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))
        model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)
        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):
        (X_train, _), (_, _) = mnist.load_data() # Load the dataset
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5 # Rescale -1 to 1
        X_train = np.expand_dims(X_train, axis=3)
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        for epoch in range(epochs):
            for _ in range(self.n_critic):
                # Train Discriminator
                idx = np.random.randint(0, X_train.shape[0], batch_size) # Select a random batch images
                imgs = X_train[idx]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim)) # Sample generator input
                d_loss = self.critic_model.train_on_batch([imgs, noise], [valid, fake, dummy])
            #  Train Generator
            g_loss = self.generator_model.train_on_batch(noise, valid)
            
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss)) # Plot the progress
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = 0.5 * self.generator.predict(noise) + 0.5 # Rescale images 0 - 1
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [3]:
if __name__ == '__main__':
    wgan = WGANGP()
    wgan.train(epochs=30000, batch_size=32, sample_interval=100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)      

KeyboardInterrupt: 

In [7]:
modelS = Sequential()
modelS.add(Dense(100, activation='relu'))

inputL = Input(shape=(100,))
x = Dense(100)(inputL)
modelA = Model(inputL, x)

type(modelS), type(modelA)
modelA( K.constant( np.random.random((1,100)) ) )

<tf.Tensor 'model_8/dense_10/BiasAdd:0' shape=(1, 100) dtype=float32>

In [ ]:
_Merge?